# **Cats or Dogs - CNN avec les méthodes de transfer**

# <a id="1">Introduction</a>  


## Dataset

La base  train contient 25 000 images de chiens et de chats. L'étiquette de chaque image de ce dossier fait partie du nom du fichier. Le dossier de test contient 12 500 images, nommées en fonction d'un identifiant numérique.
Pour chaque image de la base test, on doit prédire la probabilité que l'image soit un chien (1 = chien, 0 = chat).


## Méthode

Pour résoudre ce problème, on va utiliser un modèle pré-entraîné, ResNet-50, en remplaçant uniquement la dernière couche et le comparer avec un modèle CNN basique.

# <a id="2">Load packages</a>

In [1]:
import os, cv2, random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm
from random import shuffle 
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D,Conv2D, MaxPooling2D, Dropout
%matplotlib inline 

## Parametres

On définit ici quelques paramètres utilisés dans le modèle. La taille des images est **224**.    
Les images sont stockées dans deux dossiers, **train** et **test**.  
Il y a deux classes d'images : **Chien** et **Chat**.  
Nous allons utiliser un sous-ensemble de l'ensemble de données d'apprentissage (**20 000** images).  De l'ensemble de données d'entraînement, **50%** seront utilisés pour l'entraînement, **50%** pour la validation.  
Un modèle pré-entraîné de **ResNet-50** sera utilisé.  
Un nombre de **10** époques sera utilisé pour l'apprentissage. 




In [2]:
TEST_SIZE = 0.5
RANDOM_STATE = 2018
BATCH_SIZE = 64
NO_EPOCHS = 10
NUM_CLASSES = 2
SAMPLE_SIZE = 20000
PATH = '/kaggle/input/dogs-vs-cats-redux-kernels-edition/'
TRAIN_FOLDER = './train/'
TEST_FOLDER =  './test/'
IMG_SIZE = 224
RESNET_WEIGHTS_PATH = '/kaggle/input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

# <a id="3">Read the data</a>



In [3]:
train_image_path = os.path.join(PATH, "train.zip")
test_image_path = os.path.join(PATH, "test.zip")

In [4]:
import zipfile
with zipfile.ZipFile(train_image_path,"r") as z:
    z.extractall(".")

In [5]:
with zipfile.ZipFile(test_image_path,"r") as z:
    z.extractall(".")

En définissant la valeur SAMPLE_SIZE, on peut réduire/agrandir la taille de l'ensemble de Data train.

Actuellement, SAMPLE_SIZE est fixé à 20 000 pour éviter le problème de plantage  et consommation beaucoup de mémoire.

In [6]:
train_image_list = os.listdir("./train/")[0:SAMPLE_SIZE]
test_image_list = os.listdir("./test/")

On définit une fonction pour analyser les noms d'images afin d'extraire les 3 premières lettres des noms d'images, ce qui donne l'étiquette de l'image. Ce sera soit un chat, soit un chien. On utilise un encodeur , stockant [1,0] pour le chat et [0,1] pour le chien.

In [7]:
def label_pet_image_one_hot_encoder(img):
    pet = img.split('.')[-3]
    if pet == 'cat': return [1,0]
    elif pet == 'dog': return [0,1]

On définit également une fonction pour traiter les données (train et test). 

In [8]:
def process_data(data_image_list, DATA_FOLDER, isTrain=True):
    data_df = []
    for img in tqdm(data_image_list):
        path = os.path.join(DATA_FOLDER,img)
        if(isTrain):
            label = label_pet_image_one_hot_encoder(img)
        else:
            label = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        data_df.append([np.array(img),np.array(label)])
    shuffle(data_df)
    return data_df

# <a id="4">Data exploration</a>


## <a id="41">Class distribution</a>

On inspecte les données du train pour vérifier la distribution chat/chien.

In [9]:
def plot_image_list_count(data_image_list):
    labels = []
    for img in data_image_list:
        labels.append(img.split('.')[-3])
    sns.countplot(labels)
    plt.title('Cats and Dogs')
    

In [10]:
plot_image_list_count(os.listdir(TRAIN_FOLDER))

Tout d'abord, on traite les données Train, en lisant les images et en créant un tableau avec les images et les étiquettes. 

In [9]:
train = process_data(train_image_list, TRAIN_FOLDER)

On reproduit certaines de ces images. On commençe par une sélection de Data Train. On va montrer les 25 premières images de la Data Train.

In [12]:
def show_images(data, isTest=False):
    f, ax = plt.subplots(5,5, figsize=(15,15))
    for i,data in enumerate(data[:25]):
        img_num = data[1]
        img_data = data[0]
        label = np.argmax(img_num)
        if label  == 1: 
            str_label='Dog'
        elif label == 0: 
            str_label='Cat'
        if(isTest):
            str_label="None"
        ax[i//5, i%5].imshow(img_data)
        ax[i//5, i%5].axis('off')
        ax[i//5, i%5].set_title("Label: {}".format(str_label))
    plt.show()

show_images(train)

Et pour la base test

In [10]:
test = process_data(test_image_list, TEST_FOLDER, False)

In [14]:
show_images(test,True)

# <a id="5">Model</a>

### Prepare the train data

In [11]:
X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
y = np.array([i[1] for i in train])

## <a id="51">Basic CNN model</a>

In [12]:
def define_model():
	model = Sequential()
	model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(32, (3, 3),activation='relu'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(512, activation='relu'))
	model.add(Dense(2, activation='sigmoid'))
	# compile model
	model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
	return model

In [13]:
# define model
model = define_model()

### Split the train data in train and validation


In [14]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

In [15]:
train_model = model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS,
                  verbose=1,
                  validation_data=(X_val, y_val))

In [16]:
score = model.evaluate(X_val, y_val, verbose=0)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

## <a id="51">Resnet50 model</a>

### Prepare the model

On initialise le modèle **ResNet-50**, en ajoutant une dernière couche  de type **Dense**, avec la fonction d'activation **softmax**.   

On définit également la première couche du modèle comme non entraînable, car le modèle **ResNet-50** a déjà été entraîné.

In [12]:
model = Sequential()
model.add(ResNet50(include_top=False, pooling='max', weights=RESNET_WEIGHTS_PATH))
model.add(Dense(NUM_CLASSES, activation='softmax'))
# ResNet-50 model is already trained, should not be trained
model.layers[0].trainable = True

### Compile the model

On compile le modèle, en utilisant un opimizer **sgd**, la fonction de perte comme **categorical_crossentropy** et la métrique **accuracy**.

In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Model summary

On trace la description du modèle. On peut voir que le modèle **ResNet-50** représente la 1ère couche de notre modèle, de type **Model**.

In [14]:
model.summary()

On montre également la représentation graphique du modèle en utilisant **plot_model**.

In [20]:
plot_model(model, to_file='model.png')
SVG(model_to_dot(model).create(prog='dot', format='svg'))

### Split the train data in train and validation


In [15]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

## <a id="52">Train the model</a>



In [17]:
train_model = model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS,
                  verbose=1,
                  validation_data=(X_val, y_val))

## <a id="53">Validation accuracy and loss</a>

On représente la précision du train et de la validation sur le même graphique. De même, nous allons représenter la perte du train et de la validation sur le même graphique.

In [18]:
def plot_accuracy_and_loss(train_model):
    hist = train_model.history
    acc = hist['acc']
    val_acc = hist['val_acc']
    loss = hist['loss']
    val_loss = hist['val_loss']
    epochs = range(len(acc))
    f, ax = plt.subplots(1,2, figsize=(14,6))
    ax[0].plot(epochs, acc, 'g', label='Training accuracy')
    ax[0].plot(epochs, val_acc, 'r', label='Validation accuracy')
    ax[0].set_title('Training and validation accuracy')
    ax[0].legend()
    ax[1].plot(epochs, loss, 'g', label='Training loss')
    ax[1].plot(epochs, val_loss, 'r', label='Validation loss')
    ax[1].set_title('Training and validation loss')
    ax[1].legend()
    plt.show()
plot_accuracy_and_loss(train_model)

In [19]:
score = model.evaluate(X_val, y_val, verbose=0)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

## <a id="54">Validation accuracy per class</a>

On montre la précision de la validation pour chaque classe.

On commence par prédire les étiquettes de l'ensemble de validation.

In [20]:
#get the predictions for the test data
predicted_classes = model.predict_classes(X_val)
#get the indices to be plotted
y_true = np.argmax(y_val,axis=1)

On crée deux indices, **correct** et **incorrect**, pour les images de l'ensemble de validation dont la classe est prédite correctement et incorrectement, respectivement.

In [21]:
correct = np.nonzero(predicted_classes==y_true)[0]
incorrect = np.nonzero(predicted_classes!=y_true)[0]


On avons vu quel est le nombre de valeurs correctement prédites par rapport aux valeurs incorrectement prédites dans l'ensemble de validation.    

On montre ici le rapport de classification pour l'ensemble de validation, avec la précision par classe et globale.

In [22]:
target_names = ["Class {}:".format(i) for i in range(NUM_CLASSES)]
print(classification_report(y_true, predicted_classes, target_names=target_names))

# <a id="6">Prepare the submission</a>

### Show test images with predicted class

On montre quelques images de test avec la classe prédite. Pour cela, on doit prédire la classe.


In [23]:
f, ax = plt.subplots(5,5, figsize=(15,15))
for i,data in enumerate(test[:25]):
    img_num = data[1]
    img_data = data[0]
    orig = img_data
    data = img_data.reshape(-1,IMG_SIZE,IMG_SIZE,3)
    model_out = model.predict([data])[0]
    
    if np.argmax(model_out) == 1: 
        str_predicted='Dog'
    else: 
        str_predicted='Cat'
    ax[i//5, i%5].imshow(orig)
    ax[i//5, i%5].axis('off')
    ax[i//5, i%5].set_title("Predicted:{}".format(str_predicted))    
plt.show()

### Test data prediction

In [ ]:
pred_list = []
img_list = []
for img in tqdm(test):
    img_data = img[0]
    img_idx = img[1]
    data = img_data.reshape(-1,IMG_SIZE,IMG_SIZE,3)
    predicted = model.predict([data])[0]
    img_list.append(img_idx)
    pred_list.append(predicted[1])

### Submission file


In [ ]:
submission = pd.DataFrame({'id':img_list , 'label':pred_list})
submission.head()
submission.to_csv("submission.csv", index=False)

# <a id="7">Conclusions</a>

En utilisant le modèle pré-entraîné pour Keras, ResNet-50, on a pu obtenir un modèle assez bon en termes de précision de validation. 
On a obtenu accuracy=0.67
Comparant avec le modèle CNN basique, on a obtenu accuracy de validation= 0.5 et ça prend 4minutes et 33 secondes pour l'entrainer.

Le modèle a été utilisé pour prédire les classes des images de l'ensemble de test indépendant et les résultats ont été soumis pour tester la précision de la prédiction avec desnouvelles données .  



# <a id="8">References</a>

[1] Dogs vs. Cats Redux: Kernels Edition, https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition  
[2] ResNet pretrained models for Keras, https://www.kaggle.com/keras/resnet50  



